# WP2.1 - Short circuit on three-phase passive grids

This Notebook shows exemplary tests that are implemented in the sce_test/test_all_faults_and_cases.py file.

See: https://github.com/e2nIEE/pandapower/blob/sce/pandapower/test/shortcircuit/SCE_Tests/test_all_faults_and_cases.py

Therefore it must be run in an environment where the pandapower test folders can be found by the jupyter server's python environment.
This can be done by checking out the git branch sce in pandapower and locally initializing the repository by using `pip install -e .`.

Further information on running short-circuit calculations in pandapower can be found here:

https://pandapower.readthedocs.io/en/latest/shortcircuit/run.html

https://github.com/e2nIEE/pandapower/blob/sce/tutorials/shortcircuit/shortcircuit.ipynb

In [ ]:
import os
from itertools import product

from pandapower import pp_dir
from pandapower.test.shortcircuit.sce_tests.test_all_faults_and_cases import load_test_case_data, run_test_cases, compare_results

testfiles_path = os.path.join(pp_dir, 'test', 'shortcircuit', 'sce_tests')

In [ ]:
import warnings
# Ignore all future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Ignore all runtime warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

From the defined parameters for faults, cases, network names and vector_groups as well as the values for r_faults and x_faults all combinations are generated to run as a test.

In [ ]:
# Define common parameters
net_names = [
    "test_case_2_five_bus_radial_grid",
    "test_case_3_five_bus_meshed_grid",
    "test_case_4_twenty_bus_radial_grid"
]
faults = ["LLL", "LL", "LLG", "LG"]
cases = ["max", "min"]
values = [(0.0, 0.0), (5.0, 5.0)]
vector_groups = ['Dyn', 'Yyn', 'YNyn']
lv_tol_percents = [6, 10]
fault_location_buses = [0, 1, 2, 3]
is_branch_test = [False, True]

# Create parameter list
parametrize_values = list(product(faults, cases, values, lv_tol_percents, fault_location_buses, is_branch_test))

# Create parameter list with vector group
parametrize_values_vector = list(product(
    net_names, faults[:2], cases, values, lv_tol_percents, vector_groups[:1], fault_location_buses, is_branch_test
))+list(product(
    net_names, faults[2:], cases, values, lv_tol_percents, vector_groups, fault_location_buses, is_branch_test
))


total_test_count = len(parametrize_values)+len(parametrize_values_vector)
formatted_output = (
    f"Number of parameterization values: {len(parametrize_values)}\n"
    f"Number of parameterization value vectors: {len(parametrize_values_vector)}\n"
    f"Total number of tests: {total_test_count}"
)
print(formatted_output)

The `parametrize_values` list and the `parametrize_values_vector` contain combinations for the tests.
Tests for "LL" and "LLL" using different vector groups are omitted as these have no influence on the results.
There is one test using `parametrize_values` and three using `parametrize_values_vector`.
Shown above are the amount of test for each of the lists as well as the total.

In this Notebook we will only show a exemplary examples for each test case and vector group.
The test will load the expected results from the corresponding excel file (exported from commercial PowerFactory software results).
Then the calculations are run in pandapower using the parameters from the list of permutation and compared against the expected results.
Running the tests in the file below, it is possible to execute the complete list of tests.

https://github.com/e2nIEE/pandapower/blob/sce/pandapower/test/shortcircuit/SCE_Tests/test_all_faults_and_cases.py

---

## Test Case 1: Four bus radial grid

![four_bus_radial](four_bus_radial.png)

In [ ]:
net_name = "test_case_1_four_bus_radial_grid"

fault = "LLL"
case = "max"
fault_values = (0.0, 0.0)
# lv_tol_percents does not influence max case (default value is 10)
lv_tol_percents = 10
fault_location_bus = 0
is_branch_test = False
# vector group not necesarry as the grid does not contain a transformer

# loading the network and testing data
net, dataframes = load_test_case_data(net_name, fault_location_bus)

These are all the test results that were exported from PowerFactory. Depending on the parameter combination, the corresponding dataframe is selected.

In [ ]:
dataframes["bus"]["LLL_max_10"]

Let's now select the dataframe that we need in order to execute our first test case.

With the loaded network and expected results we now can run the test to receive the dataframes and the columns that should be compared.

Below is the list of columns that will be compared.

In [ ]:
columns_to_check, net_df, pf_results = run_test_cases(net, dataframes["branch" if is_branch_test else "bus"], fault, case, fault_values, lv_tol_percents, fault_location_bus, is_branch_test)
columns_to_check

In [ ]:
net_df[columns_to_check]

In [ ]:
# dataframes["bus"]["LLL_max_10"]
pf_results[columns_to_check]

As you can see the results are matching and the comparison function will pass and not throw any errors.

In [ ]:
compare_results(columns_to_check, net_df, pf_results, is_branch_test)

## Test Case 2: Five bus radial grid

![five_bus_radial](five_bus_radial.png)

In [ ]:
net_name = "test_case_2_five_bus_radial_grid"

fault = "LG"
case = "min"
fault_values = (5.0, 5.0)
lv_tol_percents = 6
fault_location_bus = 3
is_branch_test = True
vector_group = 'Dyn'

# loading the network and testing data
net, dataframes = load_test_case_data(net_name, fault_location_bus, vector_group)

In [ ]:
columns_to_check, net_df, pf_results = run_test_cases(net, dataframes["branch" if is_branch_test else "bus"], fault, case, fault_values, lv_tol_percents, fault_location_bus, is_branch_test)
columns_to_check

In [ ]:
net_df[columns_to_check]

In [ ]:
# dataframes["branch"]["LG_min_fault_6"]
pf_results[columns_to_check]

In [ ]:
compare_results(columns_to_check, net_df, pf_results, is_branch_test)

## Test Case 3: Five bus meshed grid

![five_bus_meshed](five_bus_meshed.png)

In [ ]:
net_name = "test_case_3_five_bus_meshed_grid"

fault = "LLL"
case = "min"
fault_values = (5.0, 5.0)
lv_tol_percents = 6
fault_location_bus = 2
is_branch_test = True
vector_group = 'Yyn'

# loading the network and testing data
net, dataframes = load_test_case_data(net_name, fault_location_bus, vector_group)

In [ ]:
columns_to_check, net_df, pf_results = run_test_cases(net, dataframes["branch" if is_branch_test else "bus"], fault, case, fault_values, lv_tol_percents, fault_location_bus, is_branch_test)
columns_to_check

In [ ]:
net_df[columns_to_check]

In [ ]:
# dataframes["branch"]["LLL_min_fault_6"]
pf_results[columns_to_check]

In [ ]:
compare_results(columns_to_check, net_df, pf_results, is_branch_test)

## Test Case 4: Twenty bus radial grid

![twenty_bus_grid](twenty_bus_grid.png)

In [ ]:
net_name = "test_case_4_twenty_bus_radial_grid"

fault = "LG"
case = "max"
fault_values = (0.0, 0.0)
lv_tol_percents = 10
fault_location_bus = 3
is_branch_test = False
vector_group = 'YNyn'

# loading the network and testing data
net, dataframes = load_test_case_data(net_name, fault_location_bus, vector_group)

In [ ]:
columns_to_check, net_df, pf_results = run_test_cases(net, dataframes["branch" if is_branch_test else "bus"], fault, case, fault_values, lv_tol_percents, fault_location_bus, is_branch_test)
columns_to_check

In [ ]:
net_df[columns_to_check]

In [ ]:
# dataframes["bus"]["LG_max_10"]
pf_results[columns_to_check]

In [ ]:
compare_results(columns_to_check, net_df, pf_results, is_branch_test)